# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/data_prep.ipynb)をお使い下さい。

In [ ]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

In [ ]:
!docker build -t sm-tf-nightly-gpu container/.

## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t word2vec .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。

In [ ]:
role = get_execution_role()
hyperparameters = {'batch_size': 64,'epochs': 1}

bucket = sess.default_bucket()
output_path = 's3://{}/output'.format(bucket)

data_path = {'train': 's3://{}/data'.format(bucket)}


estimator = Estimator(image_name=image_uri,
                      role=role,
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='ml.p2.xlarge',
                      output_path=output_path)

estimator.fit(data_path)